<a href="https://colab.research.google.com/github/joshba06/Object_Detection/blob/main/PreparationAndTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Readme

**Step 1**: 

This script has to be run on both, Colab (training) and the local machine (object detection). Set the variable "system_id" to 1, if running on Colab or to 0 if running on local machine

# 1. Getting Started

## 1.1 Define parameters

In [ ]:
# Choose 1, if running on colab and 0 if running on local machine
system_id = 0

# Define the objects that you would like to train the deep learning-model with below
labels = ['Engine']

# Change model url and name if model changes
pre_trained_model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
pre_trained_model_name = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
custom_model_name = 'my_ssd_mobilenet_v2_fpnlite'

img_size = (640, 640)

## 1.2 Create file and folder structure

In [ ]:
import os
import shutil

# Local machine
if system_id == 0:
    home_path = '/Users/niklas/Virtual_Environment/Version_5/projectAutonomous'
    print('Running on local machine...')
    os.chdir(home_path)   

# Google colab    
elif system_id == 1:
    
    print('Running on Google Colab...')
    
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Setup path to home directory 
    home_path = '/content/drive/MyDrive/Colab_Notebooks/Object_Detection'
    os.chdir(home_path)    

else:
    print('No operating system was defined...')

In [ ]:
import setup
files, paths = setup.createFolderStructure(labels, home_path, system_id)

In [ ]:
# Create dict with labels
dict_labels = {}
num = 1
for label in labels:
    dict_labels[label]= num
    num += 1
print(dict_labels)

## 1.3 Installing Dependencies and importing modules

In [ ]:
# Local machine
if system_id == 0:
    try:
        import cv2 as cv

        import pkg_resources
        installed_packages = pkg_resources.working_set
        installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
           for i in installed_packages])
        #print(installed_packages_list)

        for i in range(len(installed_packages_list)):
            package_name = installed_packages_list[i].split('==')[0]

            # Uninstall any package of openCV that is not "opencv-contrib-python"
            if ('opencv' in package_name) and (package_name != 'opencv-contrib-python'):
                package_name = installed_packages_list[i].split('==')[0]
                print('Uninstalling '+package_name)
                !pip uninstall {package_name} -y

            # Check if correct version of "opencv-contrib-python" is installed, otherwise re-install it
            elif package_name == 'opencv-contrib-python':
                package_version = installed_packages_list[i].split('==')[1]
                #print('Version of package "{}" is: {}'.format(package_name, package_version))
                if package_version == '4.6.0.66':
                    print('Correct package is installed:  {} version {}'.format(package_name, package_version))
                    nothing = 0
                else:
                    !pip uninstall {package_name} -y
            
        
        # Install correct version of "opencv-contrib-python" (installation is skipped if package is installed)
        !pip install opencv-contrib-python==4.6.0.66


    except:
        !pip install opencv-contrib-python==4.6.0.66

elif system_id == 1:
    print('No need to install openCV in Colab')

In [ ]:
# Package for downloading content via URLs
!pip install wget

# Package for displaying opencv in jupyter notebook plot
!pip install pyqt5
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import cv2 as cv

import uuid

import time

import pathlib

import shutil

import math

import wget

import sys

# 3. Prepare Tensorflow Object Detection API

## 3.1 Install dependencies

In [ ]:
# Local machine
if system_id == 0:
    try:
        import tensorflow as tf
        if tf.__version__ == "2.5.0":
            print('Tenforflow version 2.5.0 is already installed...')
        else:
            print('Tensorflow version is: '+str(tf.__version__))
            !pip install tensorflow==2.5.0
    except:
        !pip install tensorflow==2.5.0

# Google colab    
elif system_id == 1:
    !pip install tensorflow-gpu==2.5.0 

else:
    print('No operating system was defined...')

In [ ]:
# Download the model garden (model garden is an environment that is necessary to train new models from scratch or to continue training existing models)
# The model itself will be downloaded later

# Clone repository only if it does not exist already
os.chdir(paths['2_Tensorflow'])
if os.path.exists(paths['2_Tensorflow']+'/models/research') is False:
    print('Cloning model garden..')
    !git clone https://github.com/tensorflow/models.git
    
else:
    print('Model garden is already installed...')
    
os.chdir(paths['home'])

In [ ]:
# Install protobuf
if os.path.exists(paths['2_Tensorflow']+'/protoc/protoc-21.1-osx-aarch_64.zip') is False:

    # Go to destination directory
    os.chdir(paths['protoc'])
    protoc_url = 'https://github.com/protocolbuffers/protobuf/releases/download/v21.1/protoc-21.1-osx-aarch_64.zip'
    wget.download(protoc_url)

    # Extract all content of downloaded file
    from zipfile import ZipFile

    with ZipFile('protoc-21.1-osx-aarch_64.zip', 'r') as zipObj:
        zipObj.extractall()

    os.environ['Path'] = paths['protoc']+'/bin'
    os.chdir(paths['research'])

    !protoc object_detection/protos/*.proto --python_out=.

else:
    print('Protobuf is already installed...')
    
os.chdir(paths['home'])

In [ ]:
# Install pycocotools

# Clone repository only if it does not exist already
if os.path.exists(paths['research']+'/cocoapi') is False:
    print('Cloning cocoapi..')
    !git clone https://github.com/cocodataset/cocoapi.git
    
    # Moving cloned file to 'research' folder
    destination = paths['research']
    source = paths['home']+'/cocoapi'
    shutil.move(source, destination)
    
else:
    print('Cocoapi is already installed...')

os.chdir(paths['home'])

## 3.2 Install Tensorflow Object Detection API

In [ ]:
# Check if API has already been installed
if os.path.exists(paths['research']+'/setup.py') is False:
    print('Installing setup.py...')
    
    # Move to 'research' directory
    os.chdir(paths['research'])

    # Copy setup.py to current working directory
    !cp object_detection/packages/tf2/setup.py .

    # Execute setup.py (this command installs all dependencies needed for tf2 odapi)
    !python -m pip install .

    print('Installation complete..')

else:
    print('Object Detection API is already installed...')

# Move back to home-directory
os.chdir(paths['home'])

## 3.3 Check if API was installed successfully

In [ ]:
# Move to 'research' directory
os.chdir(paths['research'])
import object_detection

# Local machine
if system_id == 0:
    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
    

# Google colab    
elif system_id == 1:
    !pip install numpy --upgrade # This had to be added for execution on colab. Problem solved using stackoverflow

    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
else:
    print('No operating system was defined...')

# Move back to home directory
os.chdir(paths['home'])

# 4. Prepare new training job

## 4.1 Install dependencies and import modules (only on Colab)

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  ## Install missing modules for randomTrafficSign
  !pip install matplotlib
  !pip install lxml

  import numpy as np
  from xml.etree.ElementTree import ElementTree
  from xml.etree.ElementTree import Element
  import xml.etree.ElementTree as etree
  import xml.dom.minidom

  from lxml import etree
  os.chdir(paths['home'])

else:
    print('No operating system was defined...')

## 4.2 Partition images for testing and training (only on Colab)

**Important**: Images of objects must be in the following format: "Mug_1.jpg", "Cat_3.jpg" and must be located in their respective folders

In [ ]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
    !pip install -U albumentations --no-binary qudida,albumentations

In [ ]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # imagePreprocessing.py must be in same directory as this main script
    %matplotlib inline
    path = paths['0_User_Input']+'/scripts'
    os.chdir(path)
    import imageProcessing as imgprep
    os.chdir(paths['home'])

    imgprep.clearPreprocessing(paths['images'])
    imgprep.clearTraining(paths['training']+'/images')

    numImgs = 250
    upperScale = 350
    lowerScale = 150

    # Modify and multiply images and store in 1_Preprocessing folder
    imgprep.main(dict_labels, paths['backgrounds'], paths['objects'], numImgs, upperScale, lowerScale, paths['images']+'/', paths['training']+'/')

    imgprep.clearPreprocessing(paths['images'])

## 5.4 Create labelmap (only on Colab)

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # Convert label-dict to needed format
    labelmap = []
    for key in dict_labels:
      temp = {}
      temp['name'] = key
      temp['id'] = dict_labels[key]
      labelmap.append(temp)
    print(labelmap)

    files['labelmap'] = paths['annotations']+'/label_map.pbtxt'

    with open(files['labelmap'], 'w') as file:
      for label in labelmap:
          file.write('item { \n')
          file.write('\tname:\'{}\'\n'.format(label['name']))
          file.write('\tid:{}\n'.format(label['id']))
          file.write('}\n')
        
else:
    print('No operating system was defined...')

In [ ]:
# Install pandas
!pip install pandas

## 5.5 Create TFRecord (only on colab)

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Add labelmap and tfrecords to 'files' dictionary
  files['tf_train'] = paths['annotations']+'/train.record'
  files['tf_test'] = paths['annotations']+'/test.record'

  # Add line to download TF record file from nicknochnack

  # Copy generatetfrecord.py to scripts
  source = paths['0_User_Input']+'/scripts/generatetfrecord.py'
  shutil.copy(source, paths['scripts'])

  # Change directory to 'scripts'
  os.chdir(paths['workspace']+'/scripts')

  # Create / overwrite TFRecord files for training and testing

  # Create train data:
  !python generatetfrecord.py -x {paths['images_training']} -l {files['labelmap']} -o {files['tf_train']}

  # Create test data:
  !python generatetfrecord.py -x {paths['images_testing']} -l {files['labelmap']} -o {files['tf_test']}

  # Go back to home directory
  os.chdir(paths['home'])

 
else:
    print('No operating system was defined...')

## 5.6 Download pre-trained model (only on colab)



In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Update the settings for the image import and multiplication script depending on which size of image the model uses!

  # Check if the chosen model has already been downloaded
  if os.path.exists(paths['pre_trained_models']+'/'+str(pre_trained_model_name)) is False:

      # Go to destination directory
      os.chdir(paths['pre_trained_models'])
      wget.download(pre_trained_model_url)

      # Extract all content of downloaded file
      import tarfile

      file = tarfile.open('ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz')

      file.extractall(paths['pre_trained_models'])

      file.close()
                    
      # Delete downloaded tar.gz file to save storage space
      # Add code here
      #
      #
      
      # Create new folder for this model in training/models
      paths['active_model'] = paths['models']+'/'+custom_model_name
      os.makedirs(paths['active_model'])
      
      print('Model was successfully downloaded...')


  else:
    print(str(pre_trained_model_name)+' was already installed...')
    
    os.chdir(paths['home'])

else:
    print('No operating system was defined...')

## 5.7 Update the config file and pipeline for the new training job

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  ## Copy or replace pipeline in active model directory
  files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config'
  paths['active_model'] = paths['models']+'/'+custom_model_name
  files['pipeline_active'] = paths['active_model']+'/pipeline.config'
  paths['downloaded_model'] = paths['pre_trained_models']+'/'+pre_trained_model_name

  # If pipeline already exists in active directory, replace it
  if os.path.exists(files['pipeline_active']) == True:
      os.remove(files['pipeline_active'])
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline replaced in active model directory...')

  # If pipeline does not yet exist in active directory, copy it from downloaded model
  else:
      files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config' 
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline copied to active model directory...')

  ## Configure pipeline

  config = config_util.get_configs_from_pipeline_file(files['pipeline_active'])
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.io.gfile.GFile(files['pipeline_active'], "r") as f:                                                                                                                                                                                                                     
      proto_str = f.read()                                                                                                                                                                                                                                          
      text_format.Merge(proto_str, pipeline_config)  


  pipeline_config.model.ssd.num_classes = len(labels) # Number of labels the model should be trained for
  pipeline_config.train_config.batch_size = 4 # This should be the number of training jobs that run parallel

  # Get checkpoint 0 from (original) downloaded model 
  files['checkpoint0'] = paths['downloaded_model']+'/checkpoint/ckpt-0'

  pipeline_config.train_config.fine_tune_checkpoint = files['checkpoint0']

  pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

  # Get labelmap
  pipeline_config.train_input_reader.label_map_path= files['labelmap']

  # Get TF-Record
  pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['tf_train']]
  pipeline_config.eval_input_reader[0].label_map_path = files['labelmap']
  pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['tf_test']]

  config_text = text_format.MessageToString(pipeline_config)

  # Update active pipeline
  with tf.io.gfile.GFile(files['pipeline_active'], "wb") as f:                                                                                                                                                                                                                     
      f.write(config_text)   
      
  print('Pipeline successfully configured...')  

  # Copy model_main_tf2.py to workspace -> training   'TensorFlow/models/research/' file to 
  source = paths['research']+'/object_detection/model_main_tf2.py'
  destination = paths['training']
  shutil.copy(source, destination)

else:
    print('No operating system was defined...')


# 6. Start new training job (only on colab)

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  files['training_script'] = paths['training']+'/model_main_tf2.py'
  model_dir = paths['active_model']
  
  command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(files['training_script'], model_dir, files['pipeline_active'])
  # first argument: Path to the model_main_tf2.py file
  # second argument: Path to the diretory in which the pipeline.config file is placed (not the path to the file itself)
  # third argument: Path to actual pipeline.config in active directory

  # This command is necessary to fix issue with training on colab
  # source: https://stackoverflow.com/questions/70998639/dnn-library-is-not-found-ssd-mobile-net-v2-in-colab#answer-72404540
  !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

  !{command}

else:
  print('No operating system was defined...')

# 7 Evaluate training (both)

In [ ]:
# Local machine
if system_id == 0:

  paths['train'] = paths['3_Output']+'/my_ssd_mobilenet_v2_fpnlite/train'
  paths['eval'] = paths['3_Output']+'/my_ssd_mobilenet_v2_fpnlite/eval'

  os.chdir(paths['eval'])
  !tensorboard --logdir=.

# Google colab    
elif system_id == 1:

  command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(files['training_script'], model_dir, files['pipeline_active'], model_dir)
  !{command}

else:
  print('No operating system was defined...')

# 8 Download model (Colab)

In [ ]:
# Local machine
if system_id == 0:
  print('Skipping step...')

# Google colab    
elif system_id == 1:

    # Copy labelmap to model folder
    source = files['labelmap']
    shutil.copy(source, paths['active_model'])

    # Move active file directory to output folder
    from distutils.dir_util import copy_tree
    copy_tree(paths['active_model'], paths['3_Output']+'/'+custom_model_name)

    # Delete model directory from models folder
    shutil.rmtree(paths['active_model'])
    print('Removed model directory from 2_Tensorflow')

else:
  print('No operating system was defined...')